<a href="https://colab.research.google.com/github/AreebAhmad-02/Real-time-rag-pipeline-for-rag-research-papers/blob/main/distilabel_qa_synthetic_dataset_for_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install distilabel --upgrade -q

In [ ]:
# !pip install --upgrade -qqq git+https://github.com/argilla-io/distilabel.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install datasets -q -U

In [ ]:
!pip install huggingface-hub -q -U

In [ ]:
!pip install mistralai -q

# Creating hugging face dataset and data cleaning

In [ ]:
import os
import json
from datasets import Dataset

## data cleaning

In [ ]:

# Directory containing JSON files
directory = r'D:\rag prac\rag papers rag pipeline\chunks'

# Dictionary to hold data from all JSON files


def remove_newline_and_pipe(input_string):
    cleaned_string = input_string.replace('\n', ' ').replace('|', '')
    return cleaned_string


combined_data = []
system_prompt =  '''Provided the following context, I want to generate QA embedding pairs for each of my classes that are 3B and 7B LLM based on increasing complexity. For the 3B model, generate simple question-answer pairs, and for the 7B model, generate slightly more complex pairs. Please format the output in JSON as: {3B: {q1: ..., a1: ...}, {q2: ..., a2: ...}}, {7B: {q1: ..., a1: ...}, {q2: ..., a2: ...}}'''
# Iterate over all files in the directory
for filename in os.listdir(directory):
    # print(filename)
    if filename.endswith('.json'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r',encoding='utf-8') as file:
            print('filename:--------------', filename)
            data = json.load(file)
            clean_data = remove_newline_and_pipe(data['text'])
            print(type(clean_data))
            print("clean data:-----------------", clean_data)

            combined_data.append(
                {"text": clean_data, "system_prompt": system_prompt})

# Write the combined dictionary to a new JSON file
output_filepath = os.path.join(directory, 'combined_data_chunks.json')
with open(output_filepath, 'w') as output_file:
    json.dump(combined_data, output_file, indent=1)

print(f'Combined JSON saved to {output_filepath}')


In [ ]:
import json
with open('combined_data_chunks.json','r') as file:
  data_chunks_and_prompts = json.load(file)

In [ ]:

ds = Dataset.from_list(data_chunks_and_prompts)


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGr

In [ ]:
ds.push_to_hub("30ragpapersdataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/315 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Areeb-02/30ragpapersdataset/commit/4a5b426a8311d24e578041ea4868deff0f98e45d', commit_message='Upload dataset', commit_description='', oid='4a5b426a8311d24e578041ea4868deff0f98e45d', pr_url=None, pr_revision=None, pr_num=None)

# generating data set using distilabel

In [ ]:
#importing dataset
from datasets import load_dataset
dataset = load_dataset("Areeb-02/30ragpapersdataset")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'system_prompt'],
        num_rows: 1232
    })
})

In [ ]:
new_ds = dataset['train'][:10]

In [ ]:
# storing a mistral api key in environment using getpass code

from getpass import getpass

mistral_api_key = getpass('Enter your key:')
os.environ['MISTRAL_API_KEY'] = mistral_api_key


Enter your key:··········


In [ ]:
# hugging face api
# storing a mistral api key in environment using getpass code

from getpass import getpass

hf_api_key = getpass('Enter your key:')
os.environ['HF_TOKEN'] = hf_api_key

Enter your key:··········


In [ ]:
from distilabel.llms import MistralLLM
from distilabel.pipeline import Pipeline
from distilabel.steps import CombineColumns, LoadHubDataset,LoadDataFromHub, LoadDataFromDicts
from distilabel.steps.tasks import TextGeneration

from distilabel.steps.tasks.self_instruct import SelfInstruct


In [ ]:
from distilabel.llms.huggingface import InferenceEndpointsLLM

In [ ]:
!pip install mistralai -q


## dataset generation using mistralLLm api

In [ ]:
with Pipeline(name = "qa-text-generation-pipeline", description="QA text generation pipeline") as pipeline:
    # llm =  llm = InferenceEndpointsLLM(model_id="mistralai/Mistral-7B-Instruct-v0.2",)
    load_dataset = LoadDataFromHub(


        output_mappings={"text": "instruction"},  #
    )
    # load_dataset = LoadDataFromDicts(
    #     name = "load_data",
    #     data = [
    #         {
    #             "instruction":sample,
    #         } for sample in new_ds
    #     ]

    # )

    generate_with_mistral = TextGeneration( llm = MistralLLM(model="mistral-large-2402",api_key = mistral_api_key),input_batch_size = 8)

    # generate_with_mistral = TextGeneration(name = "generate_with_mistral", llm = InferenceEndpointsLLM(base_url="https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2",api_key = hf_api_key))
    load_dataset.connect(generate_with_mistral)



In [ ]:
generate_with_mistral.load()

In [ ]:


if __name__ == "__main__":
  distiset = pipeline.run(
      parameters={
          load_dataset.name: {
              "repo_id": "Areeb-02/30ragpapersdataset",
              "split": "train",
          },

          generate_with_mistral.name: {
              "llm": {
                  "generation_kwargs": {
                      "temperature": 0.2,
                      "max_new_tokens": 512,

                  }
              }
          },
      },
  )

[07/03/24 18:03:49] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to                ]8;id=42210;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=117749;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#611\611]8;;\
                             '/root/.cache/distilabel/pipelines/qa-text-generation-pipeline/b9c640810a9            
                             3b3e6cf36983eba1844d582b778b9/data'                                                   

[07/03/24 18:03:50] INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps to load...            ]8;id=893914;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=278678;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#703\703]8;;\

[07/03/24 18:03:52] INFO     ['distilabel.pipeline'] ⏳ Steps loaded: 1/2                               ]8;id=425229;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=911481;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#728\728]8;;\
                              * 'load_data_from_hub_0' workers: 0                                                  
                              * 'text_generation_0' workers: 1                                                     

[07/03/24 18:03:55] INFO     ['distilabel.pipeline'] ⏳ Steps loaded: 2/2                               ]8;id=103487;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=413321;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#728\728]8;;\
                              * 'load_data_from_hub_0' workers: 1                                                  
                              * 'text_generation_0' workers: 1                                                     

                    INFO     ['distilabel.pipeline'] ✅ All the steps have been loaded!                 ]8;id=860849;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=255052;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#732\732]8;;\

...

                    WARNING  ['distilabel.step.text_generation_0'] Subprocess traceback:               ]8;id=690010;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=637481;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py#528\528]8;;\
                                                                                                                   
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "/usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py",               
                             line 512, in _non_generator_process_loop                                              
                                 result = next(self.step.process_applying_mappings(*batch.data))                   
                               File                                                                                
                             "/usr/local/lib/python3.10/dist-packages/distilabel/steps/base.py", line              
                             512, in process_applying_mappings                                                     
                                 for output_rows in generator:                                                     
                               File                                                                                
                             "/usr/local/lib/python3.10/dist-packages/distilabel/steps/tasks/base.py",             
                             line 198, in process                                                                  
                                 outputs = self.llm.generate(                                                      
                               File "/usr/local/lib/python3.10/dist-packages/distilabel/llms/base.py",             
                             line 351, in generate                                                                 
                                 return self.event_loop.run_until_complete(                                        
                               File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line                
                             98, in run_until_complete                                                             
                                 return f.result()                                                                 
                               File "/usr/lib/python3.10/asyncio/futures.py", line 201, in result                  
                                 raise self._exception.with_traceback(self._exception_tb)                          
                               File "/usr/lib/python3.10/asyncio/tasks.py", line 234, in __step                    
                                 result = coro.throw(exc)                                                          
                               File "/usr/local/lib/python3.10/dist-packages/distilabel/llms/base.py",             
                             line 331, in _agenerate                                                               
                                 outputs = [outputs[0] for outputs in await asyncio.gather(*tasks)]                
                               File "/usr/lib/python3.10/asyncio/tasks.py", line 304, in __wakeup                  
                                 future.result()                                                                   
                               File "/usr/lib/python3.10/asyncio/tasks.py", line 232, in __step                    
                                 result = coro.send(None)                                                          
                               File                                                     

                    INFO     ['distilabel.step.text_generation_0'] 📨 Step 'text_generation_0' sending ]8;id=370451;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=974569;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py#558\558]8;;\
                             batch 153 to output queue                                                             

                    INFO     ['distilabel.step.text_generation_0'] 🏁 Finished running step            ]8;id=421241;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=756086;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/local.py#414\414]8;;\
                             'text_generation_0'                                                                   

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
distiset['default']['train'][1]

{'instruction': 'A financial analyst might query, "Which company among Google, Apple, and Nvidia reported the largest profit margins in their third-quarter reports for 2023?" or inquire about a specific company’s performance over time, such as "How does Apple’s sales trend look over the past three years?" These queries require evidence from multiple documents to formulate an answer. Due to the multifaceted nature of such queries, involving information from various sources, traditional similarity matching methods like cosine similarity between query and financial data may not be sufficient.\n\nFigure 1: RAG with multi-hop query.',
 'system_prompt': 'Provided the following context, I want to generate QA embedding pairs for each of my classes that are 3B and 7B LLM based on increasing complexity. For the 3B model, generate simple question-answer pairs, and for the 7B model, generate slightly more complex pairs. Please format the output in JSON as: {3B: {q1:, a1:}, {q2:, a2:}}, {7B: {q1:, 

In [ ]:
distiset_ds = distiset['default']['train']

In [ ]:
distiset_ds

Dataset({
    features: ['instruction', 'system_prompt', 'generation', 'distilabel_metadata', 'model_name'],
    num_rows: 1232
})

In [ ]:
distiset_ds.push_to_hub(
        "distilabel-aset-qa-30ragpapersdataset"
    )

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Areeb-02/distilabel-aset-qa-30ragpapersdataset/commit/d5055ccca512d727ae12525d2ed6dc7e0d255bd7', commit_message='Upload dataset', commit_description='', oid='d5055ccca512d727ae12525d2ed6dc7e0d255bd7', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print(distiset["default"]["train"][0]['generation'])

{
"3B": {
    "q1": "What is Retrieval-Augmented Generation (RAG) in the context of LLMs?",
    "a1": "Retrieval-Augmented Generation (RAG) is a method that optimizes the output of a large language model (LLM) by referencing an external knowledge base before generating a response. It improves the response quality and mitigates hallucinations, thereby enhancing the credibility of LLMs."
},
{
    "q2": "What is a multi-hop query in RAG applications?",
    "a2": "A multi-hop query is a user's query in real-world RAG applications that necessitates retrieving and reasoning over evidence from multiple documents."
}
},
"7B": {
    "q1": "What is the purpose of the MultiHop-RAG dataset and what does it consist of?",
    "a1": "The MultiHop-RAG dataset is developed to benchmark Retrieval-Augmented Generation (RAG) for multi-hop queries. It consists of a knowledge base, a large collection of multi-hop queries, their ground-truth answers, and the associated supporting evidence. The dataset is bui

## cleaning the response for generating the hugging face DS for 4 question

In [ ]:
def preprocess_json_string(json_string):
    # Removing newline characters and extra spaces
    print(json_string)
    print("-------------------")
    json_string = json_string.replace('\n},\n{', ',').replace('}\n},','},').replace('},\n  "q2"', ',\n"q2"')

    return json_string

In [ ]:

print(preprocess_json_string(distiset_ds['generation'][1]))

{
  "3B": {
    "q1": "Which company had the highest profit margin in Q3 2023, Google, Apple, or Nvidia?",
    "a1": "The company with the highest profit margin in Q3 2023 was Apple."
  },
  "q2": "How has Apple's sales trend been over the last three years?",
  "a2": "Apple's sales have shown a steady increase over the past three years."
  },
  "7B": {
    "q1": "Among Google, Apple, and Nvidia, which company reported the largest profit margins in their third-quarter reports for the fiscal year 2023?",
    "a1": "Apple reported the largest profit margins in their third-quarter reports for the fiscal year 2023."
  },
  "q2": "Could you provide an analysis of Apple's sales performance over the past three years?",
  "a2": "Over the past three years, Apple's sales performance has been robust, demonstrating a consistent upward trend."
  }
}
-------------------
{
  "3B": {
    "q1": "Which company had the highest profit margin in Q3 2023, Google, Apple, or Nvidia?",
    "a1": "The company wi

In [ ]:

def preprocessing_responses(distiset_ds):
  dict_3B_7B = []
  issue_responses = []
  count = 0
  for item in distiset_ds:
      try:
          # Extract the JSON string
          raw_json_string = item['generation']

          # Preprocess the JSON string
          preprocessed_json_string = preprocess_json_string(raw_json_string)

          if preprocessed_json_string is not None:
              # Load the JSON string into a dictionary
              data = json.loads(preprocessed_json_string)
              dict_3B_7B.append(data)
          else:
              print("Skipping due to preprocessing error")
      except json.JSONDecodeError as e:
          issue_responses.append([count,e])
          print(count)
          print(f"JSON decode error: {e}")
      except Exception as e:
          print(count)
          issue_responses.append([count,e])
          print(f"Unexpected error: {e}")
      count+=1
  return dict_3B_7B

dict_3B_7B = preprocessing_responses(distiset_ds)


Streaming output truncated to the last 5000 lines.
    "q1": "What is Retrieval Augmented Generation (RAG)?",
    "a1": "Retrieval Augmented Generation (RAG) is a strategy that involves finding documents that semantically match a query and then passing the documents to a large language model (LLM) to extract the right answer.",
    "q2": "What are the aims of a RAG system?",
    "a2": "The aims of a RAG system are to reduce the problem of hallucinated responses from LLMs, link sources/references to generated responses, and remove the need for annotating documents with meta-data."
},
"7B": {
    "q1": "What are the limitations of RAG systems?",
    "a1": "RAG systems suffer from limitations inherent to information retrieval systems and from reliance on LLMs.",
    "q2": "What are the two key takeaways from the experience report on the failure points of RAG systems?",
    "a2": "The two key takeaways are: 1) validation of a RAG system is only feasible during operation, and 2) the robustn

In [ ]:
dict_3B_7B[0]

{'3B': {'q1': 'What is Retrieval-Augmented Generation (RAG) in the context of LLMs?',
  'a1': 'Retrieval-Augmented Generation (RAG) is a method that optimizes the output of a large language model (LLM) by referencing an external knowledge base before generating a response. It improves the response quality and mitigates hallucinations, thereby enhancing the credibility of LLMs.',
  'q2': 'What is a multi-hop query in RAG applications?',
  'a2': "A multi-hop query is a user's query in real-world RAG applications that necessitates retrieving and reasoning over evidence from multiple documents."},
 '7B': {'q1': 'What is the purpose of the MultiHop-RAG dataset and what does it consist of?',
  'a1': 'The MultiHop-RAG dataset is developed to benchmark Retrieval-Augmented Generation (RAG) for multi-hop queries. It consists of a knowledge base, a large collection of multi-hop queries, their ground-truth answers, and the associated supporting evidence. The dataset is built using an English news 

In [ ]:
print(distiset_ds['generation'][1])

{
  "3B": {
    "q1": "Which company had the highest profit margin in Q3 2023, Google, Apple, or Nvidia?",
    "a1": "The company with the highest profit margin in Q3 2023 was Apple."
  },
  "q2": "How has Apple's sales trend been over the last three years?",
  "a2": "Apple's sales have shown a steady increase over the past three years."
  },
  "7B": {
    "q1": "Among Google, Apple, and Nvidia, which company reported the largest profit margins in their third-quarter reports for the fiscal year 2023?",
    "a1": "Apple reported the largest profit margins in their third-quarter reports for the fiscal year 2023."
  },
  "q2": "Could you provide an analysis of Apple's sales performance over the past three years?",
  "a2": "Over the past three years, Apple's sales performance has been robust, demonstrating a consistent upward trend."
  }
}


In [ ]:
len(issue_responses)
# issue_responses

222

In [ ]:
rag_papers_qa_dataset = []
for data in dict_3B_7B:
  converted_data = {
    "3B_q1": data['3B'].get('q1', ''),
    "3B_a1": data['3B'].get('a1', ''),
    "3B_q2": data['3B'].get('q2', ''),
    "3B_a2": data['3B'].get('a2', ''),
    "7B_q1": data['7B'].get('q1', ''),
    "7B_a1": data['7B'].get('a1', ''),
    "7B_q2": data['7B'].get('q2', ''),
    "7B_a2": data['7B'].get('a2', '')
    }
  rag_papers_qa_dataset.append(converted_data)



[{'3B_q1': 'What is Retrieval-Augmented Generation (RAG) in the context of LLMs?',
  '3B_a1': 'Retrieval-Augmented Generation (RAG) is a method that optimizes the output of a large language model (LLM) by referencing an external knowledge base before generating a response. It improves the response quality and mitigates hallucinations, thereby enhancing the credibility of LLMs.',
  '3B_q2': 'What is a multi-hop query in RAG applications?',
  '3B_a2': "A multi-hop query is a user's query in real-world RAG applications that necessitates retrieving and reasoning over evidence from multiple documents.",
  '7B_q1': 'What is the purpose of the MultiHop-RAG dataset and what does it consist of?',
  '7B_a1': 'The MultiHop-RAG dataset is developed to benchmark Retrieval-Augmented Generation (RAG) for multi-hop queries. It consists of a knowledge base, a large collection of multi-hop queries, their ground-truth answers, and the associated supporting evidence. The dataset is built using an English 

## converting to DF and pushing to hub

In [ ]:
rag_papers_qa_dataset_hf = Dataset.from_list(rag_papers_qa_dataset)

In [ ]:
rag_papers_qa_dataset_hf.push_to_hub("30rag_papers_qa_dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Areeb-02/30rag_papers_qa_dataset/commit/193c3a4c8229337d221c09e42274b0ea99580fe6', commit_message='Upload dataset', commit_description='', oid='193c3a4c8229337d221c09e42274b0ea99580fe6', pr_url=None, pr_revision=None, pr_num=None)